<a href="https://colab.research.google.com/github/Renan-Domingues/PyTorchRecipes/blob/main/SavingAndLoadingAcrossDevices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Saving and loading models across devices in PyTorch

There may be instances where I want to save and load your neural networks across different devices.

# Introduction

Saving and loading models across devices is relatively straightforward

We'll saving and loading models across CPUs and GPUs.

# Steps

1. Import all necessary libraries for loading our data
2. Define and initialize the neural network
3. Save on a GPU, load on a CPU
4. Save on a GPU, load on a GPU
5. Save on a CPU, load on a GPU
6. Saving and loading DataParallel models


### 1. Import the libraries

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

### 2. Define and initialize the neural network

In [10]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(x, self):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 3. Save on GPU, Load on CPU

When loading a model on a CPU that was trained with a GPU, pass torch.device('cpu') to the map_location argument in the torch.load() function.

In [11]:
# specify a path to save to

PATH = "model.pt"

torch.save(net.state_dict(), PATH)

# Load
device = torch.device('cpu')
model = Net()
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In this case, the storages underlying the tensors are dynamically remapped to the CPU device using the map_location argument.

### 4. Save on GPU, Load on GPU

When loading a model on a GPU that was trained and saved on GPU, simply convert the initialized model to a CUDA optimized model using model.to(torch.device('cuda')).

Be sure to use the .to(torch.device('cuda')) function on all model inputs to prepare the data for the model.

In [ ]:
# save
torch.save(net.state_dict(), PATH)

# Load

device = torch.device("cuda")
model = Net()
model.load_state_dict(torch.load(PATH))
model.to(device)

##### NOTE

it's not working in my Pc because I'm not using NVIDIA driver sistem

Note that calling my_tensor.to(device) returns a new copy of my_tensor on GPU. It does NOT overwrite my_tensor. Therefore, remember to manually overwrite tensors: my_tensor = my_tensor.to(torch.device('cuda')).

### 5. Save on CPU, Load on GPU

When loading a model on a GPU that was trained and saved on CPU, set the map_location argument in the torch.load() function to cuda:device_id. This loads the model to a given GPU device.

In [ ]:
# save

torch.save(net.state_dict(), PATH)

# Load

device = torch.device("cuda")
model = Net()

# Choose GPU device number
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))

model.to(device)

##### NOTE
Because I'm not using NVIDEA drive, tris code will not going to work either

### 6. Saving torch.nn.DataParallel Models

torch.nn.DataParallel is a model wrapper that enables parallel GPU utilization.

To save a DataParallel model generically, save the model.module.state_dict(). This way, I have the flexibility to load the model any way I want to any device I want.